In [3]:
import copy, random
from collections import namedtuple, deque
from math import sqrt, log, exp, inf
from itertools import count

# ignore deprecation warnings ('safe' as long as we don't update packages)
from warnings import filterwarnings
filterwarnings("ignore")

# misc
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

# go env
import gym
from gym_go.gogame import turn, valid_moves

# torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [4]:
# Global constants
UCB_C = 30
BOARD_SIZE = 3

'''
The state object that is returned by the reset and step functions of the environment is a
6 x BOARD_SIZE x BOARD_SIZE numpy array. All values in the array are either 0 or 1.
'''
BLACK = 0
WHITE = 1
TURN = 2
INVALID = 3
PASS = 4
DONE = 5

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE, komi=0, reward_method='heuristic')

In [ ]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        # input: 1 x BOARD_SIZE**2 x 6 channels
        self.conv1 = nn.Conv2d(6, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, BOARD_SIZE*BOARD_SIZE + 1)
        # output: 1 x BOARD_SIZE**2 + 1

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x    

In [ ]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def export_model(model, name="model.pth"):
    torch.save(model.state_dict(), "models/" + name)

def import_model():
    
    pass

def train_model():
    pass

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

# Monte Carlo Tree Search

1. Selection
    - Taverse the tree to find greatest UCB-score
2. Expansion
    - If the selected leaf node has been visited before expand by adding weighted game action
3. Rollout
    - Simulate the game until end-condition from the expanded leaf
4. Back-propagation
    - Updating the value of each ancestor node of the expanded leaf


In [248]:
# TODO: GymGo.gogame.move/validmove/elns for å nulle ut actionspace
def get_legal_move(env):
    board_shape = env.state().shape[1:]
    pass_id = np.prod(board_shape)
    action = env.action_space.sample() # pick random action
    action2d = action // board_shape[0], action % board_shape[1], action
    while action2d[2] != pass_id and env.state()[INVALID, action2d[0], action2d[1]] == 1:
        action = env.action_space.sample() # pick random action
        action2d = action // board_shape[0], action % board_shape[1], action
    return action2d[2]

def equals(state_a, state_b):
    for i in range(0, 6):
        if not np.array_equal(state_a[i], state_b[i]):
            return False
    return True


In [ ]:
class Node():
    def __init__(self, env, parent, action):
        self.env : gym.Env = env # This env will be altered by the other player
        self.value : int = 0 # Value estimate
        self.trials : int = 0 # Number of trials for this node
        self.parent : Node = parent # Parent node of this node
        self.children : list[Node] = [] # List of children of this node
        self.action : int = action # The step action made by this node
    
    # calculate a Upper Confidence Bound
    def ucb(self, total_trials):
        value = self.value
        if turn(self.env.state()) == BLACK:
            value = -self.value
        return value + ( UCB_C * sqrt(log(total_trials) / self.trials) )
    
    # Add a new node to a leaf node
    def expansion(self):
        if self.env.done:
            return
        board_shape = self.env.state().shape[1:]
        pass_id = np.prod(board_shape)
        for action in range(0, pass_id + 1):
            action2d = action // board_shape[0], action % board_shape[1]
            if action == pass_id or self.env.state()[3, action2d[0], action2d[1]] == 0:
                child_env = copy.deepcopy(self.env)
                child_env.step(action)
                self.children.append(Node(child_env, self, action))

    # Simulate game from current move until end-condition returning the score
    def rollout(self):
        if self.env.done:
            return self.env.reward()
        
        rollout_env = copy.deepcopy(self.env)
        rollout_result = 0
        done = False
        while not done:
            random_action = get_legal_move(rollout_env)
            _, reward, done, _ = rollout_env.step(random_action)
            rollout_result += reward
        return rollout_result

In [ ]:
class Monte_Carlo_Tree_Search():
    def __init__(self, size, komi):
        self.env : gym.Env = gym.make('gym_go:go-v0', size=size, komi=komi, reward_method='real')
        self.env.reset()
        self.number_of_trials : int = 0
        self.root = Node(self.env, None, None)
    
    # Update scores of all parent nodes after rollout
    def back_propagation(self, rollout_node: Node, rollout_result):
        current_node = rollout_node
        while current_node != None:
            current_node.trials += 1
            current_node.value += rollout_result
            current_node = current_node.parent
        self.number_of_trials += 1
    
    # find and return the leaf node with the highest UCB-score 
    def selection(self):
        selected_child = self.root
        current_node = self.root
        while len(current_node.children) > 0:
            selected_child = current_node.children[0]
            current_best_ucb = -inf

            for child in current_node.children:
                if child.trials == 0:
                    return child

                child_ucb = child.ucb(self.number_of_trials)

                if child_ucb > current_best_ucb:
                    selected_child = child
                    current_best_ucb = child_ucb

            current_node = selected_child

        return selected_child
        
    def run(self, iterations):
        selected_node = self.root
        selected_node.expansion()
        selected_node = self.root.children[0]

        run = 0
        while run < iterations:
            selected_node = self.selection()
            # print("Run:", run, ": Selection", selected_node.action)

            if selected_node.env.done:
                self.back_propagation(selected_node, selected_node.env.reward())
                run += 1
                continue

            if selected_node.trials > 0:
                # print("Run:", run, ": Expansion:")
                selected_node.expansion()
                selected_node = selected_node.children[0]

            rollout_result = selected_node.rollout()
            # print("Run:", run, ": Rollout", rollout_result)
            
            self.back_propagation(selected_node, rollout_result)
            run += 1
    
    def find_node_from_state(self, node: Node, state):
        if equals(node.env.state(), state):
            return node

        for child in node.children:
            if equals(child.env.state(), state):
                return child
            
            res = self.find_node_from_state(child, state)
            if res != None and equals(res.env.state(), state):
                return res

    def get_move_from_env(self, env):
        node = self.find_node_from_state(self.root, env.state())
        
        if node != None:
            print("Found node for state:")
            node.env.render()
            print("Finding best move from:", len(node.children), "available explored moves.")
            best_child = None
            current_best_value = -inf
            for child in node.children:
                value = child.value
                if turn(self.env.state()) == BLACK:
                    value = -child.value

                if value > current_best_value:
                    best_child = child
                    current_best_value = value

            if best_child != None:
                return best_child
        
        return None

In [ ]:
model = Monte_Carlo_Tree_Search(5,0)
model.run(1000)

In [ ]:
# Sjekker hvor utforska treet er for å teste UCB_C konstanten

def print_tried_children(current_node : Node):
    for child in current_node.children:
        if child.trials == 0:
            continue
        print("Child", child.action, ":")
        print("Number of children:", len(child.children))
        print("Value:", child.value, "Number of trials:", child.trials, "UCB:", child.ucb(model.number_of_trials))

current_node = model.root
while len(current_node.children) > 0:
    most_tried = current_node.children[0]
    for child in current_node.children:
        if child.trials == 0:
            continue
        print("Child", child.action, ":")
        child.env.render()
        print("Number of children:", len(child.children))
        print("Value:", child.value, "Number of trials:", child.trials, "UCB:", child.ucb(model.number_of_trials))
        print_tried_children(child)
        if child.trials > most_tried.trials:
            most_tried = child
        current_node = most_tried

Child 0 :
	0 1 2 3 4 
0	○═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Number of children: 25
Value: 2.0 Number of trials: 20 UCB: 19.630910003576
Child 1 :
Number of children: 0
Value: -1.0 Number of trials: 1 UCB: 79.84782654635399
Child 2 :
Number of children: 0
Value: -1.0 Number of trials: 1 UCB: 79.84782654635399
Child 3 :
Number of children: 0
Value: -1.0 Number of trials: 1 UCB: 79.84782654635399
Child 4 :
Number of children: 0
Value: 1.0 Number of trials: 1 UCB: 77.84782654635399
Child 5 :
Number of children: 0
Value: 1.0 Number of trials: 1 UCB: 77.84782654635399
Child 6 :
Number of children: 0
Value: -1.0 Number of trials: 1 UCB: 79.84782654635399
Child 7 :
Number of children: 0
Value: -1.0 Number of trials: 1 UCB: 79.84782654635399
Child 8 :
Number of children: 0
Value: 1.0 Number of trials: 1 UCB: 77.84782654635399
Child 9 :
Number of children: 0
Value: 1.0 Number of trials: 1

In [ ]:
def play_game(advesary_function, model : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        action = advesary_function(go_env)
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

        if done:
            continue

        node = model.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node != None:
            action = node.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')
        turn_nr += 1
        if turn_nr > 300:
            break

    if node != None:
        model.back_propagation(node, go_env.reward())

def play_game_no_render(advesary_function, model : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        action = advesary_function(go_env)
        _, _, done, _ = go_env.step(action)

        if done:
            continue

        node = model.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node != None:
            action = node.action  
        _, _, done, _ = go_env.step(action)
        turn_nr += 1
        if turn_nr > 300:
            break

    if node != None:
        model.back_propagation(node, go_env.reward())
    
    return go_env

def play_model_vs_model(model1 : Monte_Carlo_Tree_Search, model2 : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        node1 = model1.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node1 != None:
            action = node1.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

        if done:
            continue

        node2 = model2.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node2 != None:
            action = node2.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')
        turn_nr += 1
        if turn_nr > 300:
            break

    if node1 != None:
        model1.back_propagation(node1, go_env.reward())
    if node2 != None:
        model2.back_propagation(node2, go_env.reward())

def play_model_vs_model_no_render(model1 : Monte_Carlo_Tree_Search, model2 : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        node1 = model1.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node1 != None:
            action = node1.action  
        _, _, done, reward = go_env.step(action)

        if done:
            continue

        node2 = model2.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node2 != None:
            action = node2.action  
        _, _, done, reward = go_env.step(action)
        turn_nr += 1
        if turn_nr > 300:
            break
    return go_env

In [ ]:
for i in range(1,10):
    env = play_game_no_render(get_legal_move, model, copy.deepcopy(model.env))
    if env.done:
        print("Game finished within 300 turns:")
    else:
        print("Game stopped after 300 turns:")
        
    if env.reward() < 0:
        print("White won!")
    if env.reward() > 0:
        print("Black won!")
    if env.reward() == 0:
        print("It's a draw!")

Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─○─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 25 available explored moves.
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═●═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─○─╢
3	╟─┼─┼─┼─╢
4	╚═○═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 2, White Area: 1

Finding best move from: 0 available explored moves.
Game finished within 300 turns:
Black won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	○─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 25 available explored moves.
Game finished within 300 turns:
Black won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─○─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 25 availa